In [9]:
from pathlib import Path

import pandas as pd

import libs.commons as commons

In [10]:
train_path = Path(commons.dataset_path) / "train.csv"

train_set = pd.read_csv(train_path)

In [15]:
train_set['text_len'] = train_set['text'].apply(len)

In [17]:
train_set.head()

,id,keyword,location,text,target,text_len
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1,69
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1,38
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1,133
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1,65
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1,88


In [26]:
## Frequency of Keyword values
# train_set.describe()
# train_set['keyword'].nunique()
key_group = train_set.groupby('keyword')
key_group.count()

,id,location,text,target,text_len
keyword,,,,,
ablaze,36,29,36,36,36
accident,35,28,35,35,35
aftershock,34,25,34,34,34
airplane%20accident,35,29,35,35,35
ambulance,38,26,38,38,38
...,...,...,...,...,...
wounded,37,27,37,37,37
wounds,33,27,33,33,33
wreck,37,27,37,37,37


In [35]:
## Frequency of Keyword values
loc_group = train_set.groupby('location')
loc_group.count().sort_values(by='id', ascending=False)

,id,keyword,text,target,text_len
location,,,,,
USA,104,104,104,104,104
New York,71,71,71,71,71
United States,50,50,50,50,50
London,45,45,45,45,45
Canada,29,29,29,29,29
...,...,...,...,...,...
Hueco Mundo,1,1,1,1,1
"Hughes, AR",1,1,1,1,1
"Huntington, WV",1,1,1,1,1


In [43]:
def clean_loc(x):
    x = str(x)
    if x:
        return x.lower()
    else:
        return None

train_proc = train_set.copy()
train_proc['location_clean'] = train_proc['location'].apply(clean_loc)
loc_group = train_proc.groupby('location_clean')
loc_group.count().sort_values(by='id', ascending=False)

,id,keyword,location,text,target,text_len
location_clean,,,,,,
nan,2533,2472,0,2533,2533,2533
usa,104,104,104,104,104,104
new york,75,75,75,75,75,75
united states,50,50,50,50,50,50
london,49,49,49,49,49,49
...,...,...,...,...,...,...
"haiku, maui, hawaii",1,1,1,1,1,1
hailing from dayton,1,1,1,1,1,1
halfrica,1,1,1,1,1,1


In [ ]:
for loc in loc_group.groups.keys():
    print(loc)